In [ ]:
from gensim import utils
import jieba
import numpy as np
from gensim.models.doc2vec import *
import keras
import numpy as np
import pandas
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.svm import SVC, LinearSVC
from sklearn import neighbors
num_class = 18
import pandas as pd
pd.options.mode.chained_assignment = None
import time

######filtering out non-Chinese comments#########
#get_ipython().magic(u'cat comments/uid0.txt | grep -v "Not comment in Chinese" > comments/uid0_canto.json') # exclude non-Chinese comments comments
#get_ipython().magic(u'cat comments/all_comments | grep -v "Not comment in Chinese" > comments/all_canto.json') # exclude non-Chinese comments comments
#################################################


df_canto = pd.read_json('comments/all_comments_canto.json', lines=True)

###########remove reviews without comments
df_canto['comment'].replace('', np.nan, inplace=True)
df_canto.dropna(subset=['comment'],inplace=True)

####training data: X = comment; Y = emoji (1-5 scale)

df_15 = df_canto.loc[df_canto['worthy'].isin(['1','5'])]

train_x = df_15[['comment']]
train_y = df_15[['worthy']]

start = time.clock()


########Tokenizing Cantonese text with jieba########
dictionary = list() # segmented text
for index, sentence in train_x.iterrows():
    comment = sentence['comment']
    seg = " ".join(list(jieba.cut(comment)))
    dictionary.append([seg])

    
####put segmented text into a file#####   
with open("ALL_15_source_seg_nolabel.txt", "w") as f:
    for pair in dictionary:        
        f.write(" ".join(pair))
        f.write("\n")

documents = TaggedLineDocument("ALL_15_source_seg_nolabel.txt")
model = Doc2Vec(documents, vector_size=100, window=8, min_count=1, workers=4)


model.save("ALL_15_picc_doc2vec.vec")         
end = time.clock()
print ("Total running time: ", end-start)
print("Most similar to good: ", model.most_similar('好')) # just to test the model


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 5.178 seconds.
Prefix dict has been built succesfully.


In [4]:
#########Testing the model###########

print ("first review in original format: ", train_x.iloc[0] )
print ("Emoji rank of the first review: ", train_y.iloc[0] )
print ("first review in segmented format: ", dictionary[0] )
print ("first review in doc2vec format: ", model.docvecs[0] )



first review in original format:  comment    雖然貴. 但真心好食嘅爆汁牛肉餅. 九龍城除咗泰國菜出名外. 仲有呢間. 清真牛肉館. 佢地...
Name: 0, dtype: object
Emoji rank of the first review:  worthy    3
Name: 0, dtype: int64
first review in segmented format:  ['雖然 貴 .   但 真心 好 食 嘅 爆 汁 牛肉 餅 .   九龍城 除 咗 泰國菜 出名 外 .   仲有 呢 間 .   清真 牛肉 館 .   佢 地 最 出名 嘅 就 係 呢 個 牛肉 餅 .   細細 一件 已經要 .   價錢 確實 有 啲 貴 .   但 味道 真 係 唔 錯 .   外皮 煎 得 好 香脆 .   一咬落 去 .   啲 湯汁 噴晒出 嚟 .   如果 無經驗者 .   真 係 超 容易 整污 糟件 衫 .   其實 我 差少少 都 中招 .   好彩 身手 敏捷 .   避開 咗 .   另外 佢 肉 味 香 濃 .   口感 飽滿 .   以 牛肉 餅呢 講 .   佢 都 算 一 哥 .   如果 佢 係 旺 區 開 分店 就 好 啦 .   咁 就 唔 洗 吓 吓碌入 九龍城 先食 到']
first review in doc2vec format:  [-0.03593909 -0.08665214  0.02126475 -0.0515711   0.00361872  0.0712563
 -0.06323193 -0.03537343  0.04411558  0.1097589   0.06126192  0.04691868
 -0.0353487  -0.00806015  0.08015794 -0.12773752 -0.02744497  0.03089508
 -0.00489742  0.08140569 -0.05906105 -0.07337315 -0.01832197 -0.01317464
  0.02596863 -0.08374956 -0.02183769 -0.23581041 -0.10425872  0.05293929
  0.09718

In [10]:


num_line = len(model.docvecs)
print(num_line)

#######Put the doc2vec representation for each comment into an array
x_vecs = np.zeros((num_line, 100))
for i in range(0,num_line):
    x_vecs[i] = model.docvecs[i]

#######Put y_labels (without column names) into an array    
y_label = []
for index, label in train_y.iterrows():
    label_only = label['worthy']
    y_label.append(label_only)

######Slice the first 1000 lines to make a 
######Normally, we should have done some cross-validation
train_num = 1000
X_test = x_vecs[:train_num]
Y_test = y_label[:train_num]
X_train = x_vecs[train_num+1:]
Y_train = y_label[train_num+1:]

########
clf_logreg = LogisticRegression()
clf_logreg.fit(X_train, Y_train)
print ("Logistic regression: ", clf_logreg.score(X_test, Y_test) )
########
clf_svc = LinearSVC()
clf_svc.fit(X_train, Y_train)
print ("Support Vector Machines: ", clf_svc.score(X_test, Y_test) )
########
clf_nei = neighbors.KNeighborsClassifier(num_class, weights='distance')
clf_nei.fit(X_train, Y_train)
print ("Nearest neighbours: ", clf_nei.score(X_test, Y_test) )


from sklearn.linear_model import LogisticRegression


########
clf_logreg = LogisticRegression()
clf_logreg.fit(X_train, Y_train)
print ("Logistic regression: ", clf_logreg.score(X_test, Y_test) )
########
clf_svc = LinearSVC()
clf_svc.fit(X_train, Y_train)
print ("Support Vector Machines: ", clf_svc.score(X_test, Y_test) )
########
clf_nei = neighbors.KNeighborsClassifier(num_class, weights='distance')
clf_nei.fit(X_train, Y_train)
print ("Nearest neighbours: ", clf_nei.score(X_test, Y_test) )

